In [112]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re
from torch.autograd import Variable
import nltk
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [113]:
patterns = "[«»A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

In [114]:
df_csv = pd.read_csv(r"C:\Users\natal\lab_2_python\annotation1.csv")
texts = []
for absolute_path, rating in zip(df_csv['absolute_path'], df_csv['rating']):
    with open(absolute_path, 'r', encoding='utf-8') as file:
        text = file.read()
        texts.append((text, rating))
        
df = pd.DataFrame(texts, columns=['review', 'rating'])

In [115]:
df['rating'] = df['rating'].replace({1: 0, 2: 1, 3: 2, 4: 3, 5: 4})
print(df)

                                                 review  rating
0     Архимаг ищет невесту\nМного читала академок, р...       0
1     Манюня\nВсё, что нужно знать об уровне юмора э...       0
2     Удивительный Александр и крылатые кошки\nПочем...       0
3     Бездушный принц\nЗахотелось прочитать какую-ни...       0
4     Душа осьминога. Тайны сознания удивительного с...       0
...                                                 ...     ...
5000  Спаси меня от холода ночи\nЖизнь деревни Каслд...       4
5001  Витаминка\nКаково это быть купидоном? Это волш...       4
5002  Цветок в мужской академии магии\nХотела бы рас...       4
5003  Господин Неудача\nПосле загадочной гибели роди...       4
5004  Кай\n"Прежде чем войти куда-то, убедись, что т...       4

[5005 rows x 2 columns]


In [116]:
def lemmatize(review: str):
    review = re.sub(patterns, ' ', review)
    tokens = nltk.word_tokenize(review.lower())
    preprocessed_text = []
    for token in tokens:
        lemma = morph.parse(token)[0].normal_form
        if lemma not in stopwords_ru:
            preprocessed_text.append(lemma)
    return preprocessed_text

In [117]:
df['review'] = df['review'].apply(lemmatize)
print(df.head())

                                              review  rating
0  [архимаг, искать, невеста, читать, академка, р...       0
1  [манюнить, всё, нужно, знать, уровень, юмор, к...       0
2  [удивительный, александр, крылатый, кошка, поч...       0
3  [бездушный, принц, захотеться, прочитать, нибы...       0
4  [душа, осьминог, тайна, сознание, удивительный...       0


In [118]:
def join_lemmatized_words(words_list):
    return ' '.join(words_list)

In [119]:
df['review'] = df['review'].apply(join_lemmatized_words)

In [134]:
max_words = 10000
vectorizer = CountVectorizer(max_features=max_words, stop_words=stopwords_ru)
sparse_matrix = vectorizer.fit_transform(df['review']).toarray()
print(sparse_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [135]:
x_train, x_test_valid, y_train, y_test_valid = train_test_split(sparse_matrix, np.array(df['rating']), test_size=0.2)
x_test, x_valid, y_test, y_valid = train_test_split(x_test_valid, y_test_valid, test_size=0.5)

print(x_train.shape)


(4004, 10000)


In [137]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 1000)
        self.linear2 = nn.Linear(1000, 100)
        self.linear3 = nn.Linear(100, 5) #soft max [0.1, 0.2, 0.2, 0.4, 0.1] 

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return F.log_softmax(x, dim=1)


In [138]:
model = LogisticRegression()

In [139]:
optimizer = torch.optim.Adam(params=model.parameters() , lr=0.01)
criterion = nn.NLLLoss() 

In [140]:
x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()

In [141]:
epochs = 60
loss_values = []

model.train()

for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    loss_values.append(loss.item())

    # Оценка точности модели
    pred = y_pred.data.max(1, keepdim=True)[1]
    correct = pred.eq(y_train.data.view_as(pred)).sum()
    accuracy = 100. * correct / len(x_train)

    print('Epoch: {}, Loss: {:.6f}, Accuracy: {}/{} ({:.2f}%)'.format(
        epoch, loss.item(), correct, len(x_train), accuracy))

Epoch: 0, Loss: 1.634942, Accuracy: 817/4004 (20.40%)
Epoch: 1, Loss: 1.609973, Accuracy: 830/4004 (20.73%)
Epoch: 2, Loss: 1.533553, Accuracy: 1231/4004 (30.74%)
Epoch: 3, Loss: 1.630290, Accuracy: 1075/4004 (26.85%)
Epoch: 4, Loss: 1.411689, Accuracy: 1941/4004 (48.48%)
Epoch: 5, Loss: 1.448523, Accuracy: 1281/4004 (31.99%)
Epoch: 6, Loss: 1.332672, Accuracy: 2026/4004 (50.60%)
Epoch: 7, Loss: 1.265123, Accuracy: 2297/4004 (57.37%)
Epoch: 8, Loss: 1.221911, Accuracy: 2384/4004 (59.54%)
Epoch: 9, Loss: 1.124300, Accuracy: 2636/4004 (65.83%)
Epoch: 10, Loss: 1.071962, Accuracy: 2599/4004 (64.91%)
Epoch: 11, Loss: 0.984239, Accuracy: 2793/4004 (69.76%)
Epoch: 12, Loss: 0.901571, Accuracy: 2918/4004 (72.88%)
Epoch: 13, Loss: 0.854655, Accuracy: 2938/4004 (73.38%)
Epoch: 14, Loss: 0.779300, Accuracy: 3023/4004 (75.50%)
Epoch: 15, Loss: 0.706826, Accuracy: 3088/4004 (77.12%)
Epoch: 16, Loss: 0.657047, Accuracy: 3113/4004 (77.75%)
Epoch: 17, Loss: 0.592401, Accuracy: 3221/4004 (80.44%)
Epoc

In [142]:
x_test = Variable(torch.from_numpy(x_test)).float()
y_test = Variable(torch.from_numpy(y_test)).long()

In [143]:
model.eval()
with torch.no_grad():
    y_pred = model(x_test)
    probabilities = F.softmax(y_pred, dim=1)
    _, predicted = torch.max(probabilities, 1)
    correct = (predicted == y_test).sum().item()
    accuracy = correct / len(x_test)
    print("Accuracy: {}%".format(100 * accuracy))

Accuracy: 37.2%
